HW6.1

In [203]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
%matplotlib inline
import torch
import torch.optim as optim
import time

In [205]:
file_path = 'E:/UNCC/UNCC ML/Housing.csv'
sample = pd.DataFrame(pd.read_csv(file_path))
sample.head(3)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished


In [206]:
# Converting the strings to values

list1 =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
list2 = ['furnishingstatus']

# Defining the map function
def map1(x):
    return x.map({'yes': 1, 'no': 0})

def map2(x):
    return x.map({'furnished': 1, 'semi-furnished': 0})
    
sample[list1] = sample[list1].apply(map1)
sample[list2] = sample[list2].apply(map2)

sample.head(3)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1.0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1.0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0.0


In [207]:
X = sample.iloc[:, 1:12].values
Y = sample.iloc[:, 0].values
#X

In [210]:
t_x = torch.tensor(X).unsqueeze(1) # u=x
t_y = torch.tensor(Y).unsqueeze(1) # c=y
t_x.shape

torch.Size([545, 1, 11])

In [211]:
n_samples = t_y.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [212]:
t_x_train = t_x[train_indices].to(torch.float32)
t_y_train = t_y[train_indices].to(torch.float32)

t_x_val = t_x[val_indices].to(torch.float32)
t_y_val = t_y[val_indices].to(torch.float32)

In [213]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_x_train, t_x_val,
                  t_y_train, t_y_val):
    for epoch in range(1, n_epochs + 1):
        epoch_start_time = time.time()
        t_p_train = model(t_x_train) # <1>
        loss_train = loss_fn(t_p_train, t_y_train)

        t_p_val = model(t_x_val) # <1>
        loss_val = loss_fn(t_p_val, t_y_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()
        epoch_end_time = time.time()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}"
            f" Training time: {(epoch_end_time - epoch_start_time):.4f} seconds")

In [215]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [216]:
linear_model = nn.Linear(11, 1) # <1>

seq_model = nn.Sequential(
            nn.Linear(11, 56), # <1>
            nn.Tanh(),
            nn.Linear(56, 1)) # <2>

optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = loss_fn,
    t_x_train = t_x_train,
    t_x_val = t_x_val, 
    t_y_train = t_y_train,
    t_y_val = t_y_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 26310443270144.0000, Validation loss 25827385278464.0000 Training time: 0.0028 seconds
Epoch 1000, Training loss 26310443270144.0000, Validation loss 25827385278464.0000 Training time: 0.0010 seconds
Epoch 2000, Training loss 26310443270144.0000, Validation loss 25827385278464.0000 Training time: 0.0026 seconds
Epoch 3000, Training loss 26310443270144.0000, Validation loss 25827385278464.0000 Training time: 0.0020 seconds
Epoch 4000, Training loss 26310443270144.0000, Validation loss 25827385278464.0000 Training time: 0.0020 seconds
Epoch 5000, Training loss 26310443270144.0000, Validation loss 25827385278464.0000 Training time: 0.0010 seconds

Parameter containing:
tensor([[-0.0690, -0.0615,  0.2069, -0.2232,  0.2574,  0.2224,  0.1345,  0.0839,
         -0.0939, -0.0397,  0.1108]], requires_grad=True)
Parameter containing:
tensor([-0.1964], requires_grad=True)


In [219]:
seq_model = nn.Sequential(
            nn.Linear(11, 56), # <1>
            nn.Tanh(),
            nn.Linear(56, 128), # <2>
            nn.Tanh(),
            nn.Linear(128, 56), # <3>
            nn.Tanh(),
            nn.Linear(56, 1))

optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = loss_fn,
    t_x_train = t_x_train,
    t_x_val = t_x_val, 
    t_y_train = t_y_train,
    t_y_val = t_y_val)

print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 26310443270144.0000, Validation loss 25827385278464.0000 Training time: 0.0073 seconds
Epoch 1000, Training loss 26310443270144.0000, Validation loss 25827385278464.0000 Training time: 0.0033 seconds
Epoch 2000, Training loss 26310443270144.0000, Validation loss 25827385278464.0000 Training time: 0.0030 seconds
Epoch 3000, Training loss 26310443270144.0000, Validation loss 25827385278464.0000 Training time: 0.0025 seconds
Epoch 4000, Training loss 26310443270144.0000, Validation loss 25827385278464.0000 Training time: 0.0030 seconds
Epoch 5000, Training loss 26310443270144.0000, Validation loss 25827385278464.0000 Training time: 0.0031 seconds

Parameter containing:
tensor([[-0.0690, -0.0615,  0.2069, -0.2232,  0.2574,  0.2224,  0.1345,  0.0839,
         -0.0939, -0.0397,  0.1108]], requires_grad=True)
Parameter containing:
tensor([-0.1964], requires_grad=True)
